# Импорт библиотек

In [30]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [31]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
!pip install pymorphy2

In [33]:
import gensim
import pymorphy2
from nltk.tokenize import word_tokenize
from gensim.models.word2vec import Word2Vec
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim

# Загрузка и подготовка данных

In [35]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2024-11-06 12:35:31--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv.1’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.04s   

2024-11-06 12:35:31 (30.3 MB/s) - ‘Constraint_Train.csv.1’ saved [1253562/1253562]



In [36]:
df = pd.read_csv('Constraint_Train.csv')
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [37]:
tokenized_texts = [word_tokenize(text.lower()) for text in df.tweet]

In [38]:
model_tweets = Word2Vec(tokenized_texts, workers=4, vector_size=300, min_count=3, window=5, epochs=15)
model_tweets.wv.most_similar('people')

[('children', 0.7787102460861206),
 ('those', 0.7240562438964844),
 ('countries', 0.6847124099731445),
 ('self-isolating', 0.674718976020813),
 ('women', 0.666554868221283),
 ('returned', 0.6620727777481079),
 ('being', 0.6542808413505554),
 ('died', 0.6271176934242249),
 ('someone', 0.6258293986320496),
 ('they', 0.622168779373169)]

In [39]:
def get_text_embedding(text):
  result = []
  for word in word_tokenize(text.lower()):
    if word in model_tweets.wv:
      result.append(model_tweets.wv[word])

  if len(result):
      result = np.average(result, axis=0)
  else:
      result = np.zeros(300)
  return result

features = [get_text_embedding(text) for text in df.tweet]

# RandomForestClassifier

In [40]:
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    df.label,
                                                    test_size=0.2,
                                                    random_state=42)

In [41]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [42]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        fake       0.93      0.91      0.92       596
        real       0.93      0.94      0.93       688

    accuracy                           0.93      1284
   macro avg       0.93      0.93      0.93      1284
weighted avg       0.93      0.93      0.93      1284



# TfidfTransformer + LogisticRegression

In [43]:
X_train, X_test, y_train, y_test = train_test_split(df.tweet,
                                                    df.label,
                                                    test_size=0.2,
                                                    random_state=42)

In [44]:
model = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),
  ('clf', LogisticRegression(random_state=42))
])

model.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf', LogisticRegression(random_state=42))])

In [45]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        fake       0.93      0.91      0.92       596
        real       0.93      0.94      0.93       688

    accuracy                           0.93      1284
   macro avg       0.93      0.93      0.93      1284
weighted avg       0.93      0.93      0.93      1284



# RandomForestClassifier + небольшая предобработка текста

In [46]:
en_stopwords = stopwords.words('english')
analyzer = pymorphy2.MorphAnalyzer()

df['fix_tweet'] = df['tweet'].str.lower()
df['fix_tweet'] = df['fix_tweet'].apply(
      lambda x: ' '.join([word for word in x.split() if word not in (en_stopwords)]))

features = [get_text_embedding(text) for text in df.fix_tweet]

In [47]:
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    df.label,
                                                    test_size=0.2,
                                                    random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [48]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        fake       0.94      0.91      0.93       596
        real       0.93      0.95      0.94       688

    accuracy                           0.93      1284
   macro avg       0.93      0.93      0.93      1284
weighted avg       0.93      0.93      0.93      1284



# PyTorch

In [54]:
features = [get_text_embedding(text) for text in df.fix_tweet]
labels = [int(x == 'real') for x in df.label]

In [55]:
class CustomModel(nn.Module):
  def __init__(self):
    super(CustomModel, self).__init__()
    self.out = nn.Linear(300, 1)

  def forward(self, x):
    prediction = torch.sigmoid(self.out(x))
    return prediction

def train_one_epoch(in_data, targets, batch_size=16):
  for i in tqdm(range(0, in_data.shape[0], batch_size)):
    batch_x = in_data[i:i + batch_size].cuda()
    batch_y = targets[i:i + batch_size].cuda()
    optimizer.zero_grad()
    output = model(batch_x)
    loss = criterion(output.squeeze(), batch_y)
    loss.backward()
    optimizer.step()
  print(loss)

model = CustomModel()
model.cuda()

CustomModel(
  (out): Linear(in_features=300, out_features=1, bias=True)
)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    test_size=0.2,
                                                    random_state=42)

in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

In [57]:
for i in range(100):
  train_one_epoch(in_data, targets, 4)

100%|██████████| 1284/1284 [00:01<00:00, 1055.94it/s]


tensor(0.6994, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


100%|██████████| 1284/1284 [00:01<00:00, 1090.29it/s]


tensor(0.6907, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


100%|██████████| 1284/1284 [00:01<00:00, 1059.72it/s]


...

tensor(0.6658, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


100%|██████████| 1284/1284 [00:01<00:00, 1094.11it/s]


tensor(0.6658, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


100%|██████████| 1284/1284 [00:01<00:00, 1070.20it/s]

tensor(0.6657, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [58]:
with torch.no_grad():
    output = model(in_data_test.cuda()).squeeze(1)

result = (output.cpu() > 0.5) == targets_test
result.sum().item() / len(result)

0.8800623052959502